In [1]:
# Derivative Example

# inputs
h = 0.0001 # Tends to 0
a = 2.
b = -3.
c = 10.

d1 = a*b + c
c += h
d2 = a* b + c

print("d1", d1)
print("d2", d2)
print("slope", (d2 - d1)/h)


d1 4.0
d2 4.0001
slope 0.9999999999976694


In [36]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [37]:
class Value:

    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        return Value(data=self.data + other.data, _children=(self, other), _op="+")

    def __mul__(self, other):
        return Value(data=self.data * other.data, _children=(self, other), _op="*")


In [38]:
a = Value(2., label="a")
b = Value(-3., label="b")
c = Value(10, label="c")
e = a * b; e.label="e"
d = e + c; d.label="d"
d

Value(data=4.0)